In [2]:
!pip install pendulum

     |████████████████████████████████| 143kB 5.1MB/s 
     |████████████████████████████████| 491kB 42.8MB/s 


In [3]:
!pip install vega

     |████████████████████████████████| 2.2MB 4.9MB/s 


In [0]:
!pip install -U ipython

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate and create the PyDrive client.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Load a file by ID and print its contents.


In [0]:
id = '1TwBn5an9rPqZ09LNteo2Tax3IzCt7PyY'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('unemployment_rate_data.csv')  
#df3 = pd.read_csv('Filename.csv')
#downloaded = drive.CreateFile({'id': uploaded.get('1TwBn5an9rPqZ09LNteo2Tax3IzCt7PyY')})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [0]:
import pandas as pd
import pendulum
import altair as alt
# alt.renderers.enable('notebook')

**Custom utility functions**

In [0]:
def nicevalue(n):
    from math import floor, log10

    exponent = floor(log10(n))
    fraction = n / (pow(10, exponent))

    if fraction < 1.5:
        nicefraction = 1
    elif fraction < 3:
        nicefraction = 2
    elif fraction < 7:
        nicefraction = 5
    else:
        nicefraction = 10
    return nicefraction * pow(10, exponent)

def autoaxislimits(df, ymin=None, ymax=None, maxticks=6):
    from math import floor, ceil
    if ymin is None:
        ymin = df.min(numeric_only=True).min()
    if ymax is None:
        ymax = df.max(numeric_only=True).max()

    # Calculate "nice" range using nicevalue
    yrange = ymax - ymin
    niceyrange = nicevalue(yrange)

    # Calculate "nice" tickspacing using nicevalue
    tickinterval = nicevalue(niceyrange/(maxticks-1))

    # Calculate nice min and max using nice tickinterval
    nicemin = floor(ymin / tickinterval) * tickinterval
    nicemax = ceil(ymax / tickinterval) * tickinterval

    niceticks = [x for x in range(nicemin, nicemax + tickinterval, tickinterval)]

    return nicemin, nicemax, tickinterval, niceticks

def format_date(date_col, date_format):
    return pendulum.parse(date_col.strftime('%Y-%m-%d')).format(date_format)


In [0]:
# df = pd.read_csv('unemployment_rate_data.csv')
# df = pd.read_csv('unemployment_rate_data.csv', parse_dates=['date'], index_col=['date'], infer_datetime_format=True, keep_default_na=False)
df = pd.read_csv('unemployment_rate_data.csv', parse_dates=['date'], infer_datetime_format=True)
# df[col] = pd.to_datetime(df[col], format=format)



In [0]:
print(df.dtypes)

datetime64[ns]


In [0]:
df.tail(5)

,date,kyra,ohra,para,usra,wvra
233,2019-06-01,4.1,4,3.8,3.7,4.7
234,2019-07-01,4.3,4,3.9,3.7,4.7
235,2019-08-01,4.4,4.1,3.9,3.7,4.6
236,2019-09-01,4.4,4.2,4,3.5,4.7
237,2019-10-01,,,,3.6,


**Create a list of formatted date strings from the date index.**

In [0]:


df['date_str']df.date.to_series().apply(format_date, date_format='YYYY[-]MMM').tolist()

In [0]:
dfm = df.copy()
# Restrict the time range for charting
dfm = dfm[dfm['date'].between('2007-01','2019-09')]


dfm = dfm.reset_index(drop=True).melt('date')
print(dfm.head(5))
# dfm.columns.tolist()

# We set ymin to zero since UR can't go below zero by definition
ymin = 0
ymax = dfm.max(numeric_only=True).max()


# Calculate axis limits using pure Python functions
# We only include columns that are lines in the chart. (drop Recession column)
dfmin, dfmax, dftick, dfticks = autoaxislimits(dfm, ymin=ymin)
# st.write(dfmin, dfmax, dftick, dfticks)
# color_discrete_map = {"Joly": "blue", "Bergeron": "green", "Coderre":"red"}
colors=['#9E0000','#103C62','#93795C']
colors=['rgb(158, 0, 0)','rgb(16, 60, 98)','rgb(147, 121, 92)', 'rgb(89,152,113)']
import plotly.express as px
fig = px.line(dfm, x='date', y='value', color='variable', range_y=[dfmin, dfmax], color_discrete_sequence=colors)
fig.update_layout(plot_bgcolor='rgba(255,255,255,0)')


        date variable  value
0 2000-01-01     kyra    4.2
1 2000-02-01     kyra    4.1
2 2000-03-01     kyra    4.1
3 2000-04-01     kyra    4.1
4 2000-05-01     kyra    4.1


In [16]:
data = df.copy()
# Restrict the time range for charting
data = data[data['date'].between('2007-01','2019-09')]
data = data.reset_index(drop=True).melt('date')

alt.Chart(data).mark_line().encode(
    x='date:T',
    y='value',
    color='variable',
    tooltip=['variable','value']
).interactive()


alt.Chart(...)

In [4]:
from vega_datasets import data

source = data.population.url

alt.Chart(source).mark_boxplot(extent='min-max').encode(
    x='age:O',
    y='people:Q'
)

alt.Chart(...)

In [0]:
import IPython
IPython.__version__

'5.5.0'